In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data process

In [ ]:
import cv2
import os
from tqdm import *
import numpy
import time
import torch
from torchvision.models import resnet18, resnet34, resnet50
from sklearn.model_selection import train_test_split

In [ ]:
data_root = '/content/drive/MyDrive/Dataset/Ham10000/ham10000'
label_list = []
label_to_image_path_list = {}
label_to_int = {}
int_to_label = {}

In [ ]:
# load meta data
cnt = 0
for f1 in os.listdir(data_root):
    label = f1
    label_to_int[label] = cnt
    int_to_label[cnt] = label
    cnt += 1
    label_list.append(label)
    f2 = os.path.join(data_root, f1)
    label_to_image_path_list[label] = []
    for f3 in os.listdir(f2):
        f4 = os.path.join(f2, f3)
        label_to_image_path_list[label].append(f4)

print( 'label_to_int={0}'.format(label_to_int) )
print( 'int_to_label={0}'.format(int_to_label) )

label_to_int={'df': 0, 'akiec': 1, 'bcc': 2, 'bkl': 3, 'nv': 4, 'vasc': 5, 'mel': 6}
int_to_label={0: 'df', 1: 'akiec', 2: 'bcc', 3: 'bkl', 4: 'nv', 5: 'vasc', 6: 'mel'}


In [ ]:
# load image to color images
label_to_image_arr_list = {}
for label in label_to_image_path_list:
    label_to_image_arr_list[label] = []
    image_path_list = label_to_image_path_list[label]
    for image_path in tqdm(image_path_list[:10]):
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        label_to_image_arr_list[label].append(img)

# print load status
for label in label_to_image_arr_list:
    print('{0}: loaded {1} images, shape={2}'.format(label, len(label_to_image_arr_list[label]),
                                                    label_to_image_arr_list[label][0].shape))

100%|██████████| 10/10 [00:05<00:00,  1.86it/s]

df: loaded 10 images, shape=(450, 600, 3)
akiec: loaded 10 images, shape=(450, 600, 3)
bcc: loaded 10 images, shape=(450, 600, 3)
bkl: loaded 10 images, shape=(450, 600, 3)
nv: loaded 10 images, shape=(450, 600, 3)
vasc: loaded 10 images, shape=(450, 600, 3)
mel: loaded 10 images, shape=(450, 600, 3)


In [ ]:
# images to x_data, y_data
x_data = []
y_data = []
for label in label_to_image_arr_list:
    for img in label_to_image_arr_list[label]:
        x_data.append( img )
        y_data.append( label_to_int[label] )


x_data = numpy.array(x_data)
y_data = numpy.array(y_data)
print( 'x_data.shape={0}, y_data.shape={1}'.format(x_data.shape, y_data.shape) )

x_data.shape=(70, 450, 600, 3), y_data.shape=(70,)


# Resnet

In [ ]:
learning_rate = 0.001
class_count = len(label_to_image_arr_list)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load trained ResNet18 model
model = torch.load('/content/drive/MyDrive/Colab Notebooks/Model/1/model_resnet18_task1_1.pth')
# model = torch.load('/content/model_resnet18_task1.pth')
# Extract the features of trained ResNet18 model from the last CNN layer
model_last_cnn = torch.nn.Sequential(  *list(model.children())[:-2] )
model_last_cnn.add_module('flatten', torch.nn.Flatten())
for para in model_last_cnn.parameters():
    para.requires_grad = False

## The training and test sets are split in the ratio of 8:2

In [ ]:
n_channel = x_data.shape[-1]
n_count = x_data.shape[0]
width = x_data.shape[1]
height = x_data.shape[2]

x_data = x_data.reshape( (n_count,n_channel,width,height)  )
print('Before: x_data.shape={0}'.format(x_data.shape))

Before: x_data.shape=(70, 3, 450, 600)


In [ ]:
model_last_cnn.eval()
with torch.no_grad():
    x_data_tensor = torch.tensor(x_data, dtype=torch.float32).to(device)
    feateures = model_last_cnn(x_data_tensor)
    print(feateures.shape)
    x_data = feateures
#x_data = x_data.reshape((70, 456, 320))
print('After: x_data.shape={0}'.format(x_data.shape))


# train test split
x_train, x_test, y_train, y_test = train_test_split( x_data, y_data,
                                                    test_size=0.2, random_state=42 )
# output
print('x_train={0}, x_test={1}, y_train={2}, y_test={3}'.format(
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
) )

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
def get_acc(y_pred, y_true):
    asum = 0
    for i in range(len(y_pred)):
        if y_pred[i]==y_true[i]:
            asum += 1
    return asum/len(y_pred)

In [ ]:
# train and test for each epoch
# use these features to train a softmax model
# Linear model with softmax
class LinearSoftmaxModel(torch.nn.Module):
    def __init__(self):
        super(LinearSoftmaxModel, self).__init__()
        self.linear = torch.nn.Linear(145920, class_count)
    def forward(self, x):
        #return torch.nn.functional.softmax(self.linear(x))
        return self.linear(x)

In [ ]:
model_softmax = LinearSoftmaxModel().to(device)
# loss and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer_function = torch.optim.Adam(model_softmax.parameters(), lr=learning_rate)

In [ ]:
# train and test for each epoch
epoch = 10
batch_size = 64
loss_train_history_list = []
acc_test_list = []
model.to(device)

sum_time_cost_train = 0
for ep in range(epoch):
    i = 0
    batch_loss_list = []
    t0 = time.time()
    while i<len(y_train):
        t1 = time.time()

        # x_train_tensor_batch = x_train_tensor[i:i+batch_size]
        # y_train_tensor_batch = y_train_tensor[i:i+batch_size]
        x_train_tensor_batch = x_train[i:i+batch_size]
        y_train_tensor_batch = y_train[i:i+batch_size]
        x_train_tensor_batch = torch.tensor(x_train_tensor_batch, dtype=torch.float32).to(device)
        y_train_tensor_batch = torch.tensor(y_train_tensor_batch, dtype=torch.long).to(device)

        # STEP-01: train
        model.train()
        # predict
        #x_train_tensor_batch = x_train_tensor_batch.reshape((1,56,456,320))

        y_train_pred = model(x_train_tensor_batch)
        # loss
        loss = loss_function(y_train_pred, y_train_tensor_batch)
        batch_loss_list.append(loss)
        # gradient decent
        optimizer_function.zero_grad()
        loss.backward()
        optimizer_function.step()
        i = i+batch_size
        t2 = time.time()
        print('completed batch {0} of epoch {1}. loss={2}. train batch time cost={3}s'.format(i//batch_size, ep, loss, t2-t1))
    t3 = time.time()
    sum_time_cost_train += t3-t0

    # STEP-02: validation
    loss_ave = sum(batch_loss_list)/len(batch_loss_list)
    loss_train_history_list.append(loss_ave)

    # Test
    model.eval()
    asum = 0
    j=0
    with torch.no_grad():
        while j < len(y_test):
            x_test_batch = x_test[j:j+batch_size]
            y_test_batch = y_test[j:j+batch_size]
            x_test_tensor_batch = torch.tensor(x_test_batch, dtype=torch.float32).to(device)
            y_test_tensor_batch = torch.tensor(y_test_batch, dtype=torch.long).to(device)

            y_test_pred_batch = model(x_test_tensor_batch)
            y_test_pred_batch = y_test_pred_batch.cpu().detach().numpy()
            y_test_pred_batch = numpy.argmax(y_test_pred_batch, axis=1)

            for k in range(len(y_test_pred_batch)):
                if y_test_pred_batch[k]==y_test_batch[k]:
                    asum += 1

            j = j+batch_size

        t4 = time.time()
        acc_test = asum/len(y_test)
        print('completed test of epoch {0}. loss={1}. accuracy={2}. train one epoch time cost={3}s, test validation time cost={4}'.format(ep,loss, acc_test, t3-t0, t4-t3))
        acc_test_list.append(acc_test)
        print(acc_test_list)

print(sum_time_cost_train)

In [ ]:
x_train_tensor_batch.shape


torch.Size([1, 56, 456, 320])

In [ ]:
x_train_tensor_batch.reshape((1,56,456,320))



tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1021, 0.0000],
          [0.0000, 0.0000, 0.5832,  ..., 0.0000, 0.0000, 1.5041],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0673, 1.5487, 1.6204,  ..., 0.0000, 0.0000, 0.9824],
          [1.5499, 2.2840, 2.0099,  ..., 0.7675, 1.7061, 5.3534],
          [4.1474, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0162,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.2509, 0.0000, 0.2350],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0463, 0.0000, 0.0170]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0845,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [ ]:
# Define the folder path
dir_root = '/content/drive/MyDrive/Colab Notebooks/Results/Task 3/1/100/10'
if not os.path.exists(dir_root):
    os.makedirs(dir_root)

loss_train_history_list_txt = os.path.join(dir_root, 'loss_train_history_list.txt')
acc_test_list_txt = os.path.join(dir_root, 'acc_test_list.txt')

# Open file in write mode
with open(loss_train_history_list_txt, 'w') as file:
    for item in loss_train_history_list:
        file.write(str(item) + '\n')

with open(acc_test_list_txt, 'w') as file:
    for item in acc_test_list:
        file.write(str(item) + '\n')

In [ ]:
final_accuracy = acc_test_list[-1]
print(f'Final model accuracy: {final_accuracy:.5f}')

In [ ]:
loss_train_history_list_txt =os.path.join(dir_root, 'loss_train_history_list.txt')

x_list = []
loss_train_history_list = []
# Open file in write mode
i = 0
with open(loss_train_history_list_txt, 'r') as fr:
    for line in fr:
        i += 1
        line = line.strip()
        token = float( line.split('(')[1].split(',')[0] )
        loss_train_history_list.append(token)
        x_list.append(i)

# Plotting training loss curves
plt.figure(figsize=(10, 5))
plt.plot(x_list, loss_train_history_list, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(dir_root,'training_loss_curve.png'))
plt.show()

In [ ]:
acc_test_list_txt = os.path.join(dir_root, 'acc_test_list.txt')
x_list = []
acc_test_history_list = []
# Open file in write mode
i = 0
with open(acc_test_list_txt, 'r') as fr:
    for line in fr:
        i += 1
        line = line.strip()
        token = float( line )
        acc_test_history_list.append(token)
        x_list.append(i)


# Plotting test accuracy curves
plt.figure(figsize=(10, 5))
plt.plot(x_list, acc_test_history_list, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Test Accuracy Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(dir_root,'test_accuracy_curve.png'))
plt.show()